### Using Various Tools and Creating Agents with Shields

In [ ]:
import os

from dotenv import load_dotenv

In [ ]:
load_dotenv()

def create_http_client(use_local: bool = True):
    from llama_stack_client import LlamaStackClient

    host = os.environ["LLAMA_STACK_SERVER_HOST"]
    port = os.environ["LLAMA_STACK_SERVER_PORT"]

    if use_local:
        return LlamaStackClient(
            base_url=f"http://{host}:{port}"
        )
    
    togther_url = os.environ["TOGETHER_URL"]
    together_api_key = os.environ["TOGETHER_API_KEY"]

    client = LlamaStackClient(
        base_url=togther_url,
        provider_data = {
            "together_api_key": together_api_key
        }
    )
    
    return client

client = (
    create_http_client(use_local=True)
)  

### BRAVE SEARCH

In [16]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the terms described in the LICENSE file in
# the root directory of this source tree.

import json
from typing import Dict

import requests

from llama_stack_client.lib.agents.client_tool import ClientTool
from llama_stack_client.types.tool_def_param import Parameter


class BraveSearch:
    def __init__(self, api_key: str) -> None:
        self.api_key = api_key

    def search(self, query: str) -> str:
        url = "https://api.search.brave.com/res/v1/web/search"
        headers = {
            "X-Subscription-Token": self.api_key,
            "Accept-Encoding": "gzip",
            "Accept": "application/json",
        }
        payload = {"q": query}
        response = requests.get(url=url, params=payload, headers=headers)
        return json.dumps(self._clean_brave_response(response.json()))

    def _clean_brave_response(self, search_response, top_k=3):
        query = None
        clean_response = []
        if "query" in search_response:
            if "original" in search_response["query"]:
                query = search_response["query"]["original"]
        if "mixed" in search_response:
            mixed_results = search_response["mixed"]
            for m in mixed_results["main"][:top_k]:
                r_type = m["type"]
                results = search_response[r_type]["results"]
                if r_type == "web":
                    # For web data - add a single output from the search
                    idx = m["index"]
                    selected_keys = [
                        "type",
                        "title",
                        "url",
                        "description",
                        "date",
                        "extra_snippets",
                    ]
                    cleaned = {
                        k: v for k, v in results[idx].items() if k in selected_keys
                    }
                elif r_type == "faq":
                    # For faw data - take a list of all the questions & answers
                    selected_keys = ["type", "question", "answer", "title", "url"]
                    cleaned = []
                    for q in results:
                        cleaned.append(
                            {k: v for k, v in q.items() if k in selected_keys}
                        )
                elif r_type == "infobox":
                    idx = m["index"]
                    selected_keys = [
                        "type",
                        "title",
                        "url",
                        "description",
                        "long_desc",
                    ]
                    cleaned = {
                        k: v for k, v in results[idx].items() if k in selected_keys
                    }
                elif r_type == "videos":
                    selected_keys = [
                        "type",
                        "url",
                        "title",
                        "description",
                        "date",
                    ]
                    cleaned = []
                    for q in results:
                        cleaned.append(
                            {k: v for k, v in q.items() if k in selected_keys}
                        )
                elif r_type == "locations":
                    # For faw data - take a list of all the questions & answers
                    selected_keys = [
                        "type",
                        "title",
                        "url",
                        "description",
                        "coordinates",
                        "postal_address",
                        "contact",
                        "rating",
                        "distance",
                        "zoom_level",
                    ]
                    cleaned = []
                    for q in results:
                        cleaned.append(
                            {k: v for k, v in q.items() if k in selected_keys}
                        )
                elif r_type == "news":
                    # For faw data - take a list of all the questions & answers
                    selected_keys = [
                        "type",
                        "title",
                        "url",
                        "description",
                    ]
                    cleaned = []
                    for q in results:
                        cleaned.append(
                            {k: v for k, v in q.items() if k in selected_keys}
                        )
                else:
                    cleaned = []

                clean_response.append(cleaned)

        return {"query": query, "top_k": clean_response}


class WebSearchTool(ClientTool):
    """Tool to search web for queries"""

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.engine = BraveSearch(api_key)

    def get_name(self) -> str:
        return "web_search"

    def get_description(self) -> str:
        return "Search the web for a given query"

    def get_params_definition(self) -> Dict[str, Parameter]:
        return {
            "query": Parameter(
                name="query",
                parameter_type="str",
                description="The query to search for",
                required=True,
            )
        }

    def run_impl(self, query: str):
        return self.engine.search(query)

### LOGGING

In [ ]:
import logging

from llama_stack_client.lib.agents.client_tool import client_tool

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
@client_tool
def multiply(first_int: str, second_int: str) -> str:
    """
    Multiply two integers together
    :param first_int: The first integer
    :param second_int: The second integer
    :returns: The result of multiplying two integers
    """
    result = int(first_int) * int(second_int)
    
    return result

@client_tool
def add(first_int: str, second_int: str) -> str:
    """
    Adds two integers together
    :param first_int: The first integer
    :param second_int: The second integer
    :returns: The result of sum of two integers
    """
    result = int(first_int) + int(second_int)
    
    return result

@client_tool
def divide(first_int: str, second_int: str) -> str:
    """
    Divide two integers together
    :param first_int: The first integer
    :param second_int: The second integer
    :returns: The result of division of two integers
    """
    result = int(first_int) * int(second_int)
    
    return result

@client_tool
def exponentize(base: str, exponent: str) -> str:
    """
    Exponentizes the base value to the power of exponent value
    :param base: The base integer
    :param exponent: The exponent integer
    :returns: The result of exponentization of base to the exponent value
    """
    result = int(base) ** int(exponent)
    
    return result

In [ ]:
import yfinance as yf

@client_tool
def get_yf_ticker_data(ticker_symbol: str, start: str, end: str) -> str:
    """
        Get Yearly Closing prices for a given ticker symbol.
        
        :param ticker_symbol: The ticker symbol of the stock
        :param start: The start date of the data, eg: 2023-01-01
        :param end: The end date of the data, eg: 2023-12-31
        :returns: The yearly closing prices for the given ticker symbol
    """
    
    data = yf.download(ticker_symbol, start=start, end=end)
    data["year"] = data.index.year
    
    annual_close = data.groupby("year")["Close"].last().reset_index()
    annual_close_json = annual_close.to_json(orient="records", date_format="iso")
    
    return annual_close_json

In [ ]:
# To test the tool, comment the @client_tool decorator and run the function

# result = get_yf_ticker_data("AAPL", "2020-01-01", "2022-01-01")

# print(result)

In [ ]:
from llama_stack_client import Agent, AgentEventLogger

available_models = [
    model.identifier for model in client.models.list() if model.model_type == "llm"
]

print(available_models)

In [ ]:
model_id = available_models[0]

print(f"Selected model: {model_id}")    

In [ ]:
client_tools = [
    get_yf_ticker_data, 
    multiply, 
    add, 
    divide, 
    exponentize,
    WebSearchTool(api_key=os.environ["BRAVE_SEARCH_API_KEY"])
]

In [ ]:
available_shields = [shield.identifier for shield in client.shields.list()]

print(available_shields)

In [ ]:
selected_shield = available_shields[0]

print(f"Selected shield: {selected_shield}")

In [ ]:
agent = Agent(
    client=client,
    model=model_id,
    instructions="""
        You're a helpful AI assistant that responds to 
        User queries with relevant information and cites sources when available.
    """,
    tools=client_tools,
    # input_shields=[selected_shield],
    # output_shields=[selected_shield],
    input_shields=[],
    output_shields=[],
    enable_session_persistence=False
)

In [ ]:
session_id = agent.create_session("test-session")

print(f"Session ID: {session_id} in Agent : {agent.agent_id}")

In [ ]:
user_prompts = [
    "what was the closing price of Google Stock (Symbol: GOOG) in 2023?",
    "who was the 42nd President of USA?",
    "who was the Prime Minister in India when russia launched sputnik satellite to the space?",
    "take 3 to the power of 5 and multiply the result with the sum of twelve and 78, and finally the double the result"
]

for prompt in user_prompts:
    message = {
        "role": "user",
        "content": prompt
    }

    response = agent.create_turn(messages=[message], session_id=session_id)

    for log in AgentEventLogger().log(response):
        log.print()
        
    print("\n")